## Automação de Indicadores

In [2]:
# Importando os dados 
# Criar uma tabela para cada loja
# Criar um pasta para cada loja e salvar as planilhas na pasta Backup Arquivos Lojas. O formato do arquivo salvo deve ser por ex: 05_23_Nome da loja.xlsx 
# Calcular os indicadores para cada loja
# Montar a OnePage
# Enviar pora cada reespectivo gerente
# Criar um ranking para as lojas 
# Enviar um e-mail para a diretoria

In [3]:
#importando as bibliotecas
import pandas as pd
import os


### Parte 1 - Criação das tabelas da loja

In [4]:
#importando base de dados lojas
tabela_lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='ISO-8859-1', sep=';')
tabela_lojas.head()

,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco


In [5]:
#importando base de dados Emails
tabela_emails = pd.read_excel('Bases de Dados/Emails.xlsx')
tabela_emails.head()

,Loja,Gerente,Email
0,Iguatemi Esplanada,Helena,carlosrodriguesjfprojects+helena@gmail.com
1,Shopping Midway Mall,Alice,carlosrodriguesjfprojects+alice@gmail.com
2,Norte Shopping,Laura,carlosrodriguesjfprojects+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,carlosrodriguesjfprojects+manuela@gmail.com
4,Shopping União de Osasco,Valentina,carlosrodriguesjfprojects+valentina@gmail.com


In [6]:
#importando base de dados Vendas
tabela_vendas = pd.read_excel('Bases de Dados/Vendas.xlsx')
tabela_vendas.head()

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71


In [7]:
# Criando uma nova tabela de Vendas contendo o nome da loja
tabela_vendas_completa = tabela_lojas.merge(tabela_vendas, how='inner', on=['ID Loja'])
tabela_vendas_completa

,ID Loja,Loja,Código Venda,Data,Produto,Quantidade,Valor Unitário,Valor Final
0,1,Iguatemi Esplanada,1,2019-01-01,Sapato Estampa,1,358,358
1,1,Iguatemi Esplanada,1,2019-01-01,Camiseta,2,180,360
2,1,Iguatemi Esplanada,1,2019-01-01,Sapato Xadrez,1,368,368
3,1,Iguatemi Esplanada,21,2019-01-02,Camisa Gola V Listrado,2,116,232
4,1,Iguatemi Esplanada,34,2019-01-02,Sapato Listrado,1,363,363
...,...,...,...,...,...,...,...,...
100994,25,Shopping Barra,69908,2019-12-25,Meia Liso,2,38,76
100995,25,Shopping Barra,69908,2019-12-25,Camisa,1,100,100
100996,25,Shopping Barra,69913,2019-12-25,Gorro,1,80,80
100997,25,Shopping Barra,69913,2019-12-25,Cinto Linho,1,248,248


### Parte 2 - Criando um pasta, uma a tabela para cada loja e salvando os respectivos arquivos backup

In [8]:
# criando uma lista das lojas 
lista_lojas = list(tabela_lojas['Loja'])

In [9]:
# Criando uma pasta para cada loja
for loja in lista_lojas:
    
    if not os.path.isdir(f"Backup Arquivos Lojas\\{loja}"):
        os.mkdir(f'Backup Arquivos Lojas\\{loja}')


In [10]:
# Criando uma tabela para cada loja

dicionario_lojas = {}

for loja in lista_lojas:
    dicionario_lojas[loja] = tabela_vendas_completa.loc[tabela_vendas_completa['Loja'] == loja]


In [11]:
# Definindo a data do último dia 
data_atual = tabela_vendas_completa['Data'].max().strftime("%d/%m/%Y")
dia_atual = data_atual[:2]
mes_atual = data_atual[3:5]


In [12]:
# Salvando os arquivos de backup nas respectivas 
for loja in dicionario_lojas.keys():

    # Verificando se a pasta já existe
    if not os.path.isdir(f"Backup Arquivos Lojas\\{loja}"):

        os.mkdir(f'Backup Arquivos Lojas\\{loja}')

    # Salvando o arquivos de da respectiva pasta
    dicionario_lojas[loja].to_excel(f'Backup Arquivos Lojas/{loja}/{mes_atual}_{dia_atual}_{loja}.xlsx', index=False)


### Parte 3 - Cálculo dos indicadores

#### Calculo do Faturamento -> Meta Ano: 1.650.000 / Meta Dia: 1000

In [13]:
# Preparando as tabelas para cálculo por locada loja

vendas_loja_ano = dicionario_lojas['Norte Shopping']
vendas_loja_dia = vendas_loja_ano[vendas_loja_ano['Data'] == data_atual]

In [14]:
# Calculo faturamento por ano
faturamento_ano = vendas_loja_ano['Valor Final'].sum()
print(faturamento_ano)

# Calculo faturamento por dia
faturamento_dia = vendas_loja_dia['Valor Final'].sum()
print(faturamento_dia)

1711968
1259


#### Calculo da Diversidade de Produtos (quantos produtos diferentes foram vendidos naquele período) -> Meta Ano: 120 / Meta Dia: 4

In [15]:
#Calculo diversidade

qtde_produtos_ano = len(vendas_loja_ano['Produto'].unique())
print(qtde_produtos_ano)

qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
print(qtde_produtos_dia)

120
2


####  Calculo do Ticket Médio por Venda -> Meta Ano: 500 / Meta Dia: 500

In [16]:
# Ticket médio por ano
valor_venda = vendas_loja_ano[['Código Venda','Produto','Quantidade',"Valor Final"]].groupby(['Código Venda']).sum()
ticket_medio_ano = valor_venda['Valor Final'].mean()

# Ticket médio por dia
valor_venda = vendas_loja_dia[['Código Venda','Produto','Quantidade',"Valor Final"]].groupby(['Código Venda']).sum()
ticket_medio_dia = valor_venda['Valor Final'].mean()

print(ticket_medio_ano)
print(ticket_medio_dia)


784.5866177818515
1259.0


In [17]:
# Faturamento anual por loja
dicionario_faturamento_ano = {}

for loja in lista_lojas:
    faturamento_ano = dicionario_lojas[loja]['Valor Final'].agg('sum')
    dicionario_faturamento_ano[loja] = faturamento_ano
print(dicionario_faturamento_ano)


{'Iguatemi Esplanada': 1699681, 'Shopping Midway Mall': 1590441, 'Norte Shopping': 1711968, 'Shopping Iguatemi Fortaleza': 1674824, 'Shopping União de Osasco': 1663770, 'Shopping Center Interlagos': 1640486, 'Rio Mar Recife': 1722766, 'Salvador Shopping': 1646739, 'Rio Mar Shopping Fortaleza': 1698430, 'Shopping Center Leste Aricanduva': 1682870, 'Ribeirão Shopping': 1707122, 'Shopping Morumbi': 1586444, 'Parque Dom Pedro Shopping': 1631630, 'Bourbon Shopping SP': 1726110, 'Palladium Shopping Curitiba': 1721120, 'Passei das Águas Shopping': 1649014, 'Center Shopping Uberlândia': 1668921, 'Shopping Recife': 1624879, 'Shopping Vila Velha': 1731167, 'Shopping SP Market': 1721763, 'Shopping Eldorado': 1648111, 'Shopping Ibirapuera': 1635060, 'Novo Shopping Ribeirão Preto': 1678225, 'Iguatemi Campinas': 1762419, 'Shopping Barra': 1653103}


In [18]:
# Melhores e piores faturamentos e melhores e piores lojas por ano
maior_faturamento_ano = max(dicionario_faturamento_ano.values())
menor_faturamento_ano = min(dicionario_faturamento_ano.values())
melhor_loja_ano = [loja for loja, faturamento in dicionario_faturamento_ano.items() if faturamento == maior_faturamento_ano][0]
pior_loja_ano = [loja for loja, faturamento in dicionario_faturamento_ano.items() if faturamento == menor_faturamento_ano][0]

print(f'Melhor loja do ano: {melhor_loja_ano} -> R$ {maior_faturamento_ano:,.2f}')
print(f'Pior loja do ano: {pior_loja_ano} -> R$ {menor_faturamento_ano:,.2f}')

Melhor loja do ano: Iguatemi Campinas -> R$ 1,762,419.00
Pior loja do ano: Shopping Morumbi -> R$ 1,586,444.00


In [19]:
# Melhores e piores faturamentos e melhores e piores lojas por dia
tabela_vendas_dia = tabela_vendas_completa[tabela_vendas_completa['Data'] == data_atual]
maior_faturamento_dia = max(tabela_vendas_dia['Valor Final'])
menor_faturamento_dia = min(tabela_vendas_dia['Valor Final'])

melhor_loja_dia = tabela_vendas_dia.loc[tabela_vendas_dia['Valor Final'].idxmax(), 'Loja']
pior_loja_dia = tabela_vendas_dia.loc[tabela_vendas_dia['Valor Final'].idxmin(), 'Loja']

print(f'Melhor loja do dia: {melhor_loja_dia} -> R$ {maior_faturamento_dia:,.2f}')
print(f'Pior loja do dia: {pior_loja_dia} -> R$ {menor_faturamento_dia:,.2f}')

Melhor loja do dia: Salvador Shopping -> R$ 2,160.00
Pior loja do dia: Novo Shopping Ribeirão Preto -> R$ 78.00


In [20]:
# Criando o ranking anual

ranking_ano = tabela_vendas_completa[['Loja','Valor Final']].groupby(['Loja']).sum().sort_values('Valor Final', ascending= False)
display(ranking_ano.head())

,Valor Final
Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763


In [21]:
# Criando o ranking por dia

ranking_dia = tabela_vendas_dia[['Loja','Valor Final']].groupby(['Loja']).sum().sort_values('Valor Final', ascending = False)

In [22]:
# Definindo as metas

# Meta Faturamento -> Meta Ano: 1.650.000 / Meta Dia: 1000
# Calculo da Diversidade de Produtos (quantos produtos diferentes foram vendidos naquele período) -> Meta Ano: 120 / Meta Dia: 4
# Calculo do Ticket Médio por Venda -> Meta Ano: 500 / Meta Dia: 500
meta_faturamento_ano = 1650000
meta_faturamento_dia = 1000
meta_qtde_ano = 120
meta_qtde_dia = 4
meta_ticket_ano = 500
meta_ticket_dia = 500

In [23]:
# Enviando o e-mail para o Gerente

import smtplib
from email.message import EmailMessage
import imghdr


# Configurações de login
EMAIL_ADDRESS = 'carlosrodriguesjfprojects@gmail.com'
EMAIL_PASSWORD = 'gowiehdprfeamidu'

# automatizando a cor do indicador dia

if faturamento_dia >= meta_faturamento_dia:
    cor_faturamento_dia = 'green'   
else:
    cor_faturamento_dia = 'red'

if qtde_produtos_dia >= meta_qtde_dia:
    cor_qtde_dia = 'green'
else:
    cor_qtde_dia = 'red'

if ticket_medio_dia >= meta_ticket_dia:
    cor_ticket_dia = 'green'
else:
    cor_ticket_dia = 'red'

  
# automatizando a cor do indicador ano

if faturamento_ano >= meta_faturamento_ano:
    cor_faturamento_ano = 'green'   
else:
    cor_faturamento_ano = 'red'

if qtde_produtos_ano >= meta_qtde_ano:
    cor_qtde_ano = 'green'
else:
    cor_qtde_ano = 'red'

if ticket_medio_ano >= meta_ticket_ano:
    cor_ticket_ano = 'green'
else:
    cor_ticket_ano = 'red'



# Criar e enviar um email
mail = EmailMessage()
mail['Subject'] = f'OnePage dia {dia_atual}/{mes_atual}/{data_atual[7:]} Loja - {loja}' 
mensagem = f'''

<p> Bom dia,  </p>

<p>O resultado de <strong> ontem (dia {dia_atual}/{mes_atual}) </strong> da <strong> Loja {loja} </strong>foi:</p>

<table border="1">
  <tr>
    <th>Indicador</th>
    <th>Valor Dia</th>
    <th>Meta Dia</th>
    <th>Cenário Dia</th>
  </tr>
  <tr>
    <td>Faturamento</td>
    <td style="text-align: center">R$ {faturamento_dia:.2f}</td>
    <td style="text-align: center">R$ {meta_faturamento_dia:.2f}</td>
    <td style="text-align: center"><font color={cor_faturamento_dia}>◙</font></td>
  </tr>
  <tr>
    <td>Diversidade de Produtos</td>
    <td style="text-align: center">{qtde_produtos_dia}</td>
    <td style="text-align: center">{meta_qtde_dia}</td>
    <td style="text-align: center"><font color={cor_qtde_dia}>◙</font></td>
  </tr>
  <tr>
    <td>Ticket Médio</td>
    <td style="text-align: center">R$ {ticket_medio_dia:.2f}</td>
    <td style="text-align: center">R$ {meta_ticket_dia:.2f}</td>
    <td style="text-align: center"><font color={cor_ticket_dia}>◙</font></td>
  </tr>
</table>
<br>
<br>
<table border="1">
  <tr>
    <th>Indicador</th>
    <th>Valor Ano</th>
    <th>Meta Ano</th>
    <th>Cenário Ano</th>
  </tr>
  <tr>
    <td>Faturamento</td>
    <td style="text-align: center">R$ {faturamento_ano:.2f}</td>
    <td style="text-align: center">R$ {meta_faturamento_ano:.2f}</td>
    <td style="text-align: center"><font color={cor_faturamento_ano}>◙</font></td>
  </tr>
  <tr>
    <td>Diversidade de Produtos</td>
    <td style="text-align: center">{qtde_produtos_ano}</td>
    <td style="text-align: center">{meta_qtde_ano}</td>
    <td style="text-align: center"><font color={cor_qtde_ano}>◙</font></td>
  </tr>
  <tr>
    <td>Ticket Médio</td>
    <td style="text-align: center">R$ {ticket_medio_ano:.2f}</td>
    <td style="text-align: center">R$ {meta_ticket_ano:.2f}</td>
    <td style="text-align: center"><font color={cor_ticket_ano}>◙</font></td>
  </tr>
</table>


<p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>
<p>Qualquer dúvida, estou à disposição.</p>

Att.,

Fulano


'''
mail['From'] = EMAIL_ADDRESS
mail['To'] = tabela_emails.loc[tabela_emails['Loja'] == loja,'Email']
mail.add_header('Content-Type', 'text/html')
mail.set_payload(mensagem.encode('utf-8'))


# Anexar qualquer tipo de arquivo(que não seja imagem)
arquivos = [f'Backup Arquivos Lojas/{loja}/{mes_atual}_{dia_atual}_{loja}.xlsx']

for arquivo in arquivos:
    with open(arquivo, 'rb') as arquivo:
        dados = arquivo.read()
        nome_arquivo = arquivo.name
        mail.add_attachment(dados, maintype='application',
                            subtype='octet-stream', filename=nome_arquivo)

# Enviar o email
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as email:
    email.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
    email.send_message(mail)